In [1]:
'''
make_submission() generates predictions for the Kaggle Painter by Numbers competion
using simple features (image size, aspect ratio and bits/pixel^2)
author: Swaroop Krothapalli - extended code of small yello duck
https://github.com/swaroop7/painters
'''
import random
import time
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.cross_validation import KFold
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score  
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from scipy.stats import itemfreq
from sklearn import neighbors, linear_model
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import cross_validation
np.set_printoptions(precision=3, linewidth=100)

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

os.chdir('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features')

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

input_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\train_pilot_conv5_1.txt')
output_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\train_pilot_processed_conv5_1.txt','w')

random = {}

for line in input_file:
    new_line = ''
    print len(random)
    for each_word in line.split():
        words = each_word.strip().split(":")
        if ':' in each_word:
            if words[0] not in random.keys():
                random[words[0]] = 0.

print 'part 1 done'
for line in input_file:
    for each_word in line.split():
        words = each_word.strip().split(":")
        if ':' in each_word:
            random[words[0]] = round(float(words[1]), 4)
            
    for key,value in random.items():
        output_file.write(str(value)+' ')
        random[key] = 0.
    output_file.write('\n')
    
input_file.close()
output_file.close()


'\ninput_file = open(\'C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\train_conv5_1.txt\')\noutput_file = open(\'C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\train_processed_conv5_1_v1.txt\',\'w\')\n\nrandom = {}\n\nfor line in input_file:\n    new_line = \'\'\n    print len(random)\n    for each_word in line.split():\n        words = each_word.strip().split(":")\n        if \':\' in each_word:\n            if words[0] not in random.keys():\n                random[words[0]] = 0.\n\nprint \'part 1 done\'\nfor line in input_file:\n    for each_word in line.split():\n        words = each_word.strip().split(":")\n        if \':\' in each_word:\n            random[words[0]] = round(float(words[1]), 4)\n            \n    for key,value in random.items():\n        output_file.write(str(value)+\' \')\n        random[key] = 0.\n    output_file.write(\'\n\')\n    print \'line done\'\ninput_file.close()\noutput_file.close()\n'

In [ ]:

input_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\test_conv5_1.txt')
output_file = open('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\test_processed_conv5_1.txt','w')

for line in input_file:
    for each_word in line.split():
        words = each_word.strip().split(":")
        if ':' in each_word and words[0] in random.keys():
            random[words[0]] = round(float(words[1]), 4)
            
    for key,value in random.items():
        output_file.write(str(value)+' ')
        random[key] = 0.
    print 'line done'
    output_file.write('\n')
    
input_file.close()
output_file.close()


'\ninput_file = open(\'C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\test_conv5_1.txt\')\noutput_file = open(\'C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\test_processed_conv5_1_v1.txt\',\'w\')\n\nfor line in input_file:\n    for each_word in line.split():\n        words = each_word.strip().split(":")\n        if \':\' in each_word:\n            random[words[0]] = round(float(words[1]), 4)\n            \n    for key,value in random.items():\n        output_file.write(str(value)+\' \')\n        random[key] = 0.\n    output_file.write(\'\n\')\n    \ninput_file.close()\noutput_file.close()\n'

In [18]:
x_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\train_processed_conv5_1_v2.txt')
y_train = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_train_conv5_1.txt')
x_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\test_processed_conv5_1_v2.txt')
y_test = np.loadtxt('C:\\Users\\swaroop\\Downloads\\painters\\cnn_features\\y_test_conv5_1.txt')

print x_train.shape

rand_numbers = []

for x in range(1,500):
    rand_numbers.append(random.randint(1,93827))
    
new_x_train = x_train[:, rand_numbers]

new_x_test = x_test[:, rand_numbers]

print new_x_train.shape
print new_x_test.shape

(48L, 94072L)
(48L, 499L)
(10L, 499L)


In [19]:
def print_results(clf, y_test, y_pred, y_pred_prob):
    #y_pred_prob = clf.predict_proba(y_test)[:,1]
    #y_pred = clf.predict(y_test)
    print 'ROC - ',roc_auc_score(y_test, y_pred_prob)
    print 'Accuracy - ',accuracy_score(y_test, y_pred)
    print 'Confusion Matrix - ', confusion_matrix(y_test, y_pred)
    #print 'Precision - ',precision_score(y_test, y_pred ),'Recall - ',recall_score(y_test, y_pred),'F1- Score',f1_score(y_test, y_pred),'\n'
    target_names = ['class 0', 'class 1']
    print classification_report(y_test, y_pred, target_names=target_names)

In [20]:
print x_train.shape, y_train.shape
#training, testing, y_training, y_testing = train_test_split(new_x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
actual_training, validation, y_actual_training, y_validation = train_test_split(new_x_train, y_train, test_size=0.2, stratify=y_train, random_state=42)
#print training.shape, testing.shape
#print y_training, y_testing
#print itemfreq(y_training)
#print itemfreq(y_testing)
print itemfreq(y_validation)
#print itemfreq(y_train)

print y_validation
# print y_actual_training
# print actual_training


(48L, 94072L) (48L,)
[[ 0.  6.]
 [ 1.  4.]]
[ 0.  0.  1.  0.  0.  1.  1.  1.  0.  0.]


In [22]:
#Random Forest
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"        

for i in xrange(1):
    clf = RandomForestClassifier(n_estimators=80, class_weight='balanced',  random_state = 42)
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, scores.mean(), scores.std(), accuracy_score(y_validation, y_pred_valid), accuracy_score(y_test, clf.predict(new_x_test))
    
print_results(clf, y_test, clf.predict(new_x_test), clf.predict_proba(new_x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 0.696031746032 0.163492063492 0.7 0.6
ROC -  0.479166666667
Accuracy -  0.6
Confusion Matrix -  [[5 1]
 [3 1]]
             precision    recall  f1-score   support

    class 0       0.62      0.83      0.71         6
    class 1       0.50      0.25      0.33         4

avg / total       0.57      0.60      0.56        10



In [21]:
#XGB
    
learning_r_col = [0.0001, 0.001, 0.01, 0.02, 0.03, 0.04,0.05,0.06,0.07,0.08,0.09,0.1]    
    
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"        

    
for i in xrange(1):
    clf = xgb.XGBClassifier(max_depth=4, n_estimators=5, learning_rate=0.001) #objective='multi:softprob'
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, "\t",scores.mean(),"\t", scores.std(), "\t",accuracy_score(y_validation, y_pred_valid), "\t",accuracy_score(y_test, clf.predict(new_x_test))

print_results(clf, y_test, clf.predict(new_x_test), clf.predict_proba(new_x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 	0.66746031746 	0.192652906423 	0.7 	0.6
ROC -  0.6875
Accuracy -  0.6
Confusion Matrix -  [[5 1]
 [3 1]]
             precision    recall  f1-score   support

    class 0       0.62      0.83      0.71         6
    class 1       0.50      0.25      0.33         4

avg / total       0.57      0.60      0.56        10



In [24]:
#Logistic
    
print "ACCURACY RESUTLS"    
print "Mean Accuracy of 5 fold CV","\t", "STD","\t", "Validation ","\t", "Test"    
    
for i in xrange(1):  # , max_iter = i
    clf = linear_model.LogisticRegression( class_weight='balanced')
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5)
    y_pred_valid = clf.predict(validation)
    print i, scores.mean(), scores.std(), accuracy_score(y_validation, y_pred_valid), accuracy_score(y_test, clf.predict(new_x_test))
    
print_results(clf, y_test, clf.predict(new_x_test), clf.predict_proba(new_x_test)[:,1] )

ACCURACY RESUTLS
Mean Accuracy of 5 fold CV 	STD 	Validation  	Test
0 0.721825396825 0.121407820615 0.7 0.7
ROC -  0.458333333333
Accuracy -  0.7
Confusion Matrix -  [[5 1]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.71      0.83      0.77         6
    class 1       0.67      0.50      0.57         4

avg / total       0.70      0.70      0.69        10



In [26]:
#Linear SVC
    
print "ACCURACY RESUTLS of 5 fold CV"    
print "Mean Accuracy ","\t", "STD","\t", "Validation ","\t", "Test"    

iter = [10,100,500,1000]

for i in iter:
    clf = LinearSVC(class_weight = 'balanced', dual = False, max_iter = i)
    clf.fit(actual_training, y_actual_training)
    scores = cross_validation.cross_val_score(clf, actual_training, y_actual_training, cv=5 )
    y_pred_valid = clf.predict(validation)
    print i, "\t",scores.mean(),"\t", scores.std(), "\t",accuracy_score(y_validation, y_pred_valid), "\t",accuracy_score(y_test, clf.predict(new_x_test))

target_names = ['class 0', 'class 1']
print 'Confusion Matrix - ', confusion_matrix(y_test, clf.predict(new_x_test))
print classification_report(y_test, clf.predict(new_x_test), target_names=target_names)

ACCURACY RESUTLS of 5 fold CV
Mean Accuracy  	STD 	Validation  	Test
10 	0.721825396825 	0.121407820615 	0.7 	0.7
100 	0.721825396825 	0.121407820615 	0.7 	0.7
500 	0.721825396825 	0.121407820615 	0.8 	0.7
1000 	0.721825396825 	0.121407820615 	0.8 	0.7
Confusion Matrix -  [[5 1]
 [2 2]]
             precision    recall  f1-score   support

    class 0       0.71      0.83      0.77         6
    class 1       0.67      0.50      0.57         4

avg / total       0.70      0.70      0.69        10

